In [1]:
from transformers import pipeline
from PIL import Image, ImageChops
import requests
import numpy as np
import scipy
import cv2
import os

IMG_PATH =  os.path.expanduser("~/Desktop/profile.jpg")

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [193]:
# load pipe
pipe = pipeline(task="depth-estimation", model="LiheYoung/depth-anything-large-hf")

# load image
image = Image.open(IMG_PATH)

# inference
depth = pipe(image)["depth"]
depth.save(os.path.expanduser("~/Downloads/depth.jpg"))

In [194]:
from moviepy.editor import ImageSequenceClip
from IPython.display import display

In [44]:
def shift_image(img, depth_img, shift_amount=10):
    # Ensure base image has alpha
    img = img.convert("RGBA")
    data = np.array(img)

    # Ensure depth image is grayscale (for single value)
    depth_img = depth_img.convert("L")
    depth_data = np.array(depth_img)
    deltas = np.array((depth_data / 255.0) * float(shift_amount), dtype=int)

    # This creates the transprent resulting image.
    # For now, we're dealing with pixel data.
    shifted_data = np.zeros_like(data)

    width = img.width
    height = img.height

    for y, row in enumerate(deltas):
        width = len(row)
        x = 0
        while x < width:
            dx = row[x]
            if x+dx >= width:
                break
            if x-dx < 0:
                shifted_data[y][x-dx] = [0,0,0,0]
            else:
                shifted_data[y][x-dx] = data[y][x]
            x += 1

    # Convert the pixel data to an image.
    shifted_image = Image.fromarray(shifted_data)

    alphas_image = Image.fromarray(scipy.ndimage.binary_fill_holes(ImageChops.invert(shifted_image.getchannel("A")))).convert("1")
    shifted_image.putalpha(ImageChops.invert(alphas_image))
    return shifted_image

In [188]:
def shift_and_inpaint(img, path, amount):
    shifted = shift_image(img, depth, shift_amount=amount).save(path)
    org_img = cv2.imread(filename=path)
    damaged_img = cv2.imread(filename=path)
    print(damaged_img.shape)
    
    # get the shape of the image
    height, width = damaged_img.shape[0], damaged_img.shape[1]
     
    # Converting all pixels greater than zero to black while black becomes white
    for i in range(height):
        for j in range(width):
            if damaged_img[i, j].sum() > 0:
                damaged_img[i, j] = 0
            else:
                damaged_img[i, j] = [255, 255, 255]
     
    # saving the mask 
    mask = cv2.cvtColor(damaged_img, cv2.COLOR_BGR2GRAY)
    dst = cv2.inpaint(org_img, mask, 3, cv2.INPAINT_NS)
    return np.array(dst)

In [191]:
def create_stereoscopic_frame(image):
    left_img = shift_and_inpaint(image, "left.png", 10)
    right_img = shift_and_inpaint(image, "right.png", 50)
    
    height, width, channels = left_img.shape
    
    sbs_image = cv2.vconcat([left_img, right_img])  # Combine images side by side
    sbs_image = cv2.cvtColor(sbs_image, cv2.COLOR_BGR2RGB)

    # Create a video
    fps = 30  # Frames per second
    duration = 10  # Duration of the video in seconds
    clip = ImageSequenceClip([sbs_image for _ in range(fps * duration)], fps=fps)

     # Export the video
    video_path = 'sbs_3d_video.mp4'
    clip.write_videofile(video_path, codec='libx264', audio=False)

In [192]:
create_stereoscopic_frame(image)
# display(image)

(2048, 1357, 3)
(2048, 1357, 3)
Moviepy - Building video sbs_3d_video.mp4.
Moviepy - Writing video sbs_3d_video.mp4



Moviepy - Done !
Moviepy - video ready sbs_3d_video.mp4
